In [15]:
%pip install transformers datasets evaluate ipywidgets scikit-learn numpy pandas tf-keras
%pip install tensorflow[and-cuda] 
%pip install torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 26.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 68.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 45.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 86.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 92.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 97.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 48.4 MB/s eta 0

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token has not been saved to git credential helper.


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import TFBertForSequenceClassification, TFTrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import json


In [4]:
df = pd.read_json('https://raw.githubusercontent.com/Wikidepia/SQuAD-id/refs/heads/master/data/train-SQuAD-id.json')


In [5]:
squad_data = []

for index, row in df.iterrows():
    entry = row['data']
    title = entry['title']
    for paragraph in entry['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            id_ = qa['id']
            if qa['is_impossible']:
                answers = {'answer_start': [], 'text': []}
            else:
                answers = {
                    'answer_start': [answer['answer_start'] for answer in qa['answers']],
                    'text': [answer['text'] for answer in qa['answers']]
                }
            squad_data.append({
                'answers': answers,
                'context': context,
                'id': id_,
                'question': question,
                'title': title
            })

squad_data = squad_data[:200]
squad_data[0]

{'answers': {'answer_start': [299], 'text': ['pada akhir 1990-an']},
 'context': 'Beyoncé Giselle Knowles-Carter (/ biːˈjɒnseɪ / bee-YON-say) (lahir 4 September 1981) adalah seorang penyanyi, penulis lagu, produser rekaman dan aktris Amerika. Dilahirkan dan dibesarkan di Houston, Texas, ia tampil di berbagai kompetisi menyanyi dan menari sebagai seorang anak, dan mulai terkenal pada akhir 1990-an sebagai penyanyi utama dari grup wanita R&B Destiny\'s Child. Dikelola oleh ayahnya, Mathew Knowles, grup ini menjadi salah satu grup wanita terlaris di dunia sepanjang masa. Jeda mereka melihat perilisan album debut Beyoncé, Dangerously in Love (2003), yang menjadikannya sebagai artis solo di seluruh dunia, meraih lima Grammy Awards dan menampilkan single nomor satu Billboard Hot 100 "Crazy in Love" dan "Baby Boy" .',
 'id': '56be85543aeaaa14008c9063',
 'question': 'Kapan Beyonce mulai populer?',
 'title': 'Beyoncé'}

In [6]:
# Preprocessing function for refined squad_data
def preprocess_squad_data(squad_data):
    contexts = []
    questions = []
    answers = []
    
    for entry in squad_data:
        context = entry['context']
        question = entry['question']
        
        # Ambil jawaban pertama untuk keperluan training
        answer = entry['answers']['text'][0] if entry['answers']['text'] else None
        
        if answer:
            contexts.append(context)
            questions.append(question)
            answers.append(answer)
            
    return contexts, questions, answers

contexts, questions, answers = preprocess_squad_data(squad_data)


In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

tokenized_inputs = tokenizer(contexts[:2], padding=True, truncation=True, return_tensors="tf")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
W0000 00:00:1731603997.762129   19037 gpu_device.cc:2433] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 5.2. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
W0000 00:00:1731603997.763633   19037 gpu_device.cc:2433] TensorFlow was not built with CUDA kernel binaries compatible with compute capability 5.2. CUDA kernels will be jit-compiled from PTX, which could take 30 minutes or longer.
I0000 00:00:1731603997.884549   19037 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 195 MB memory:  -> device: 0, name: Tesla M60, pci bus id: 0001:00:00.0, compute capability: 5.2
